In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets
from torch.utils.data import DataLoader, Dataset
import ssl
from torchvision.transforms import transforms
from torchvision import utils
import torch.nn.functional as F
import torch.nn as nn

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")   #push to gpu to speed up computations.
print(device)
train_data = datasets.CIFAR10(root="./Cifar10", download=True, train=True, transform=transforms.ToTensor())
test_data = datasets.CIFAR10(root="./Cifar10_test", download=True, train=False, transform=transforms.ToTensor())


cuda


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./Cifar10/cifar-10-python.tar.gz to ./Cifar10


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./Cifar10_test/cifar-10-python.tar.gz to ./Cifar10_test


In [3]:

#data transformation block
meanRGB = [np.mean(x.numpy(), axis=(1,2)) for x, _ in train_data]
meanstd = [np.std(x.numpy(), axis=(1,2)) for x, _ in train_data]

meanR = np.mean([m[0] for m in meanRGB])
meanG = np.mean([m[1] for m in meanRGB])
meanB = np.mean([m[2] for m in meanRGB])

stdR = np.std([m[0] for m in meanRGB])
stdG = np.std([m[1] for m in meanRGB])
stdB = np.std([m[2] for m in meanRGB])

stats = ((meanR, meanG, meanB), (stdR,stdG, stdB))

train_transforms = transforms.Compose([transforms.ToTensor(),
                                       transforms.RandomHorizontalFlip(p=0.5),
                                       transforms.RandomVerticalFlip(p=0.5),
                                       transforms.Normalize(*stats, inplace=True)])


train_data.transform = train_transforms

In [4]:

#sample data into batches and shuffle

train_sampler = DataLoader(train_data, batch_size=64, shuffle=True)
test_sampler = DataLoader(test_data, batch_size=64, shuffle=False)

#view images

x_iter = iter(train_sampler)
X_batch, y_batch = next(x_iter)

#print(X_batch.shape, y_batch.shape)

def _view_image(img, y=None):
    npimg = img.numpy()
    npimg_tr = np.transpose(npimg, (1,2,0))
    plt.imshow(npimg_tr)
    if y is not None:
        plt.title("label: " + str(y))

    plt.show()


#grid_size = 12
#rnd_inds = np.random.randint(0, len(X_batch), grid_size)

#x_grid = [X_batch[i] for i in rnd_inds]   #samples
#y_grid = [y_batch[i] for i in rnd_inds]   #labels   issue is -> stores as class number

#x_grid = utils.make_grid(x_grid, nrow=4, ncol=3, padding=2)
#print(x_grid.shape)

#_view_image(x_grid)


In [5]:
#model
class CNN_1(nn.Module):
    def __init__(self, in_channels, n_classes):
        super(CNN_1, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 6, kernel_size=(2,2))
        self.pool1 = nn.MaxPool2d(kernel_size=(2,2), stride=1)
        self.conv2 = nn.Conv2d(6, 12, kernel_size=(2,2))
        self.pool2 = nn.MaxPool2d(kernel_size=(2,2), stride=1)
        #linear block
        self.linear1 = nn.Linear(in_features=12*28*28, out_features=150)
        self.linear2 = nn.Linear(150, out_features=n_classes)

    def forward(self, x):
        out = self.conv1(x)
        out = self.pool1(out)
        out = F.relu(out)
        out = self.conv2(out)
        out = self.pool2(out)
        out = F.relu(out)
        #print(out.shape)
        out = torch.flatten(out, start_dim=1, end_dim=-1)
        #print(out.shape)
        out = self.linear1(out)
        out = F.relu(out)
        out = self.linear2(out)
        return out


In [10]:

in_channels = 3
n_classes = 10
model = CNN_1(in_channels=in_channels, n_classes = n_classes).to(device)
epochs = 20
learning_rate = 0.001
optimizer = torch.optim.SGD(lr=learning_rate, params=model.parameters())
criterion = torch.nn.CrossEntropyLoss()


#training loop
for epoch in range(epochs):
    print(f"current epoch:{epoch+1}")
    for i , (image, label) in enumerate(train_sampler):
        image = image.to(device)
        label = label.to(device)
        out = model(image)
        loss = criterion(out, label)

        optimizer.zero_grad()
        loss.backward()

        #update
        optimizer.step()


current epoch:1
current epoch:2
current epoch:3
current epoch:4
current epoch:5
current epoch:6
current epoch:7
current epoch:8
current epoch:9
current epoch:10
current epoch:11
current epoch:12
current epoch:13
current epoch:14
current epoch:15
current epoch:16
current epoch:17
current epoch:18
current epoch:19
current epoch:20


In [11]:

def accuracy(loader, model):
    num_correct = 0
    num_samples = 0

    #no gradient computation
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            out = model(x)
            _, preds = out.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.shape[0]

    acc = 100 * num_correct / num_samples
    print(f"{num_correct} / {num_samples} were classified correctly. {acc:.3f}%")


accuracy(train_sampler, model)   #45% acc currently with 10 epochs / 50,3% with 20 epochs.



25155 / 50000 were classified correctly. 50.310%


In [ ]:
accuracy(test_sampler, model)